In [ ]:
# Prerequisites
import json
import os
from eye_ai.eye_ai import EyeAI

import pandas as pd
from pathlib import Path, PurePath
import logging
from IPython.display import Markdown, display

from deriva_ml import DatasetBag, Workflow, ExecutionConfiguration
from deriva_ml import MLVocab as vc
from deriva_ml.deriva_definitions import ColumnDefinition, BuiltinTypes
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

In [ ]:
# Login
from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
host = 'www.eye-ai.org'
# host = 'dev.eye-ai.org'
catalog_id = "eye-ai"

gnl = GlobusNativeLogin(host=host)
if gnl.is_logged_in([host]):
    print("You are already logged in.")
else:
    gnl.login([host], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")

In [ ]:
cache_dir = '/data'
working_dir = '/data'
EA = EyeAI(hostname = host, catalog_id = catalog_id, cache_dir= cache_dir, working_dir=working_dir)

# Configuration

In [ ]:
# Workflow instance
cur_workflow = Workflow(
    name="Create feature angle and laterality",
    url="https://github.com/informatics-isi-edu/eye-ai-exec/blob/main/notebooks/multimodal/Ingest_angle_laterality.ipynb",
    workflow_type="multimodal_feature_engineering"
)
# Configuration instance.

# Set to False if you only need the metadata from the bag, and not the assets.
download_assets = True

config = ExecutionConfiguration(
    # Comment out the following line if you don't need the assets.
    # datasets=[source_dataset] if download_assets else [],
    assets=['4-3S3G'],
    workflow=cur_workflow,
    description="Create feature angle and laterality on prod")

# Initialize execution
execution = EA.create_execution(config)



In [ ]:
print(execution)

# Create Feature

In [ ]:
column_left = ColumnDefinition(name='Left_Prob', type=BuiltinTypes.float4, nullok=True)
column_right = ColumnDefinition(name='Right_Prob', type=BuiltinTypes.float4, nullok=True)

feature_image_laterality = EA.create_feature(target_table="Image",
                                        feature_name = "Fundus_Laterality", 
                                        terms = ['Image_Side'],
                                        metadata = [column_left, column_right],
                                        comment="Lateraliity of fundus images")

feature_image_angle = EA.create_feature(target_table="Image",
                                             feature_name = "Fundus_Angle",
                                             terms = ["Image_Angle"], 
                                             comment="Angle of fundus images")

# Retrive a feature

In [ ]:
# Find all the features that attached to a table
table_name = "Image"
EA.find_features(table_name)

In [ ]:
Feature_lat = EA.feature_record_class(table_name, 'Fundus_Laterality')
Feature_angle = EA.feature_record_class(table_name, 'Fundus_Angle')
display(
    Markdown('### Fundus_Laterality'),
    [ f'Column: {c.name}, Required: {not c.nullok}' for c in Feature_lat.feature.feature_columns],
    Markdown('### Fundus_Angle'),
    [ f'Column: {c.name}, Required: {not c.nullok}' for c in Feature_angle.feature.feature_columns]
)

# Ingest new values to a feature table

In [ ]:
feature = pd.read_csv(execution.asset_paths[0])
feature.dropna(subset=['Image'], inplace=True)
feature.drop_duplicates(subset=['Image'], inplace=True)
feature['Angle'] = feature['Angle'].astype(str)
lat_feature_list = [
    Feature_lat(
        Execution=execution.execution_rid,
        **{
            "Image": row['Image'] if 'Image' in row and not pd.isna(row['Image']) else None,
            "Image_Side": row['Side'] if 'Side' in row and not pd.isna(row['Side']) else None,
            "Left_Prob": row['Left_Prob'] if 'Left_Prob' in row and not pd.isna(row['Left_Prob']) else None,
            "Right_Prob": row['Right_Prob'] if 'Right_Prob' in row and not pd.isna(row['Right_Prob']) else None,
        },
        Feature_Name='Fundus_Laterality'
    )
    for index, row in feature.iterrows()
]

angle_feature_list = [Feature_angle(
    Execution=execution.execution_rid,
    Image= row['Image'],
    Image_Angle = row['Angle'],
    Feature_Name = 'Fundus_Angle'
    ) for index, row in feature.iterrows()]

In [ ]:
lat_feature_list = [
    Feature_lat(
        Execution=execution.execution_rid,
        **{
            "Image": row['Image'] if 'Image' in row and not pd.isna(row['Image']) else None,
            "Image_Side": row['Side'] if 'Side' in row and not pd.isna(row['Side']) else None,
            "Left_Prob": row['Left_Prob'] if 'Left_Prob' in row and not pd.isna(row['Left_Prob']) else None,
            "Right_Prob": row['Right_Prob'] if 'Right_Prob' in row and not pd.isna(row['Right_Prob']) else None,
        },
        Feature_Name='Fundus_Laterality'
    )
    for index, row in feature.iterrows()
]

In [ ]:
lat_feature_list

In [ ]:
# Ingest new values to the feature table
EA.add_features(angle_feature_list)
EA.add_features(lat_feature_list)



# Upload results

In [ ]:
# save assets to asset_path

# upload assets to catalog
execution.upload_execution_outputs(clean_folder=True)